In [23]:
%matplotlib notebook
#%matplotlib inline # notebook, qt, wx, gtk, osx, tk, empty uses default
import numpy as np
import argparse
import imutils
import cv2
from matplotlib import pyplot as plt

ap = argparse.ArgumentParser()

In [24]:
image = cv2.imread("IMG_20181216_092615.jpg")
img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(img)